### Модуль предобработки данных. 

In [1]:
#pip install scikit-learn==1.1
#%pip install --upgrade scikit-learn==1.4.0
#

In [1]:
import pandas as pd
import os
import pickle
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from numpy import genfromtxt
from tempfile import TemporaryFile

##### Имитируем подачу данных с датчиков

In [2]:
General_path = "C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/"
raw = "raw/"
processed = "processed/"
final = "final/"
tests = "tests/"

# train_FD001 = pd.read_csv(General_path + "train_FD001.csv")
# train_FD002 = pd.read_csv(General_path + "train_FD002.csv")
# train_FD003 = pd.read_csv(General_path + "train_FD003.csv")
# train_FD004 = pd.read_csv(General_path + "train_FD004.csv")

# test_FD001 = pd.read_csv(General_path + "test_FD001.csv")
# test_FD002 = pd.read_csv(General_path + "test_FD002.csv")
# test_FD003 = pd.read_csv(General_path + "test_FD003.csv")
# test_FD004 = pd.read_csv(General_path + "test_FD004.csv")

# This dataset using for show preprocess handling incoming data
# BAD_test_FD001 = pd.read_csv(General_path + "BAD_test_FD001.csv")

# Small_BAD_test_FD001 = pd.read_csv(General_path + raw + "Small_BAD_test_FD001.csv")
# np_Small_BAD_test_FD001 = genfromtxt(General_path + raw + "Small_BAD_test_FD001.csv", delimiter=',')
# np_Small_GOOD_test_FD001 = genfromtxt(General_path + raw + "Small_GOOD_test_FD001.csv", delimiter=',')

New_train_FD001 = genfromtxt(General_path + processed + "New_train_FD001.csv", delimiter=',')
train_FD001 = genfromtxt(General_path + raw + "train_FD001.csv", delimiter=',')
# Small_GOOD_test_FD001 = pd.read_csv(General_path + "Small_GOOD_test_FD001.csv")

##### Проверим датасет

In [3]:
print("np_Small_BAD_test_FD001:")
print(np_Small_BAD_test_FD001)
print("\nnp_Small_GOOD_test_FD001:")
print(np_Small_GOOD_test_FD001)

np_Small_BAD_test_FD001:
[[     nan      nan      nan      nan]
 [ 1.0e+00  1.0e+00      nan  3.0e-04]
 [ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]
 [ 1.0e+00  3.0e+00  3.0e-04      nan]
 [ 1.0e+00      nan  4.2e-03  0.0e+00]]

np_Small_GOOD_test_FD001:
[[ 1.0e+00  1.0e+00  2.3e-03  3.0e-04]
 [ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]
 [ 1.0e+00  3.0e+00  3.0e-04  1.0e-04]
 [ 1.0e+00  4.0e+00  4.2e-03  0.0e+00]]


In [10]:
GOOD_dataFrame = np.array([[1, 1, 1],
                           [1, 1, 1],
                           [1, 1, 1]])

### Поиск пропусков в pd.Dataframe

In [ ]:
# def is_null(dataframe: pd.DataFrame):
#     count_nulls = 0
#     null_set = pd.Dadataframe(dataframe.isnull())

#     for col in dataframe.columns:
#         for val in dataframe[col].is_null():
#             if (val == True):
#                 count_nulls +=1
#             #print(val)
#         print("in ", col, "nulls: ", count_nulls)
#         count_nulls = 0

# is_null(np_Small_BAD_test_FD001)

### Создание и сохранение Pipeline

#### Создаем SimpleInputer - простой заполнитель пропущенных значений в датасете

Так как предполагается, что данные упорядоченны по циклам работы двигателя, не целесообразно использовать стандартный заполнитель SimpleImputer. разумнее будет вычислить среднее значение лишь до и после пропущенного значения в каждом столбце. Можно использовать KNNImputer.

In [16]:
simple_inputer = KNNImputer(n_neighbors = 2)
std_scaler = StandardScaler()

pipe_num = Pipeline([('imputer', simple_inputer),('scaler', std_scaler)])

#### Обучаем Pipeline

In [13]:
res_num = pipe_num.fit_transform(np_Small_BAD_test_FD001)
res_num

array([[ 0.        ,  0.        , -0.01370506,  0.15075567],
       [ 0.        , -1.58113883,  0.05482024,  1.6583124 ],
       [ 0.        ,  0.        , -1.52126178, -1.35680105],
       [ 0.        ,  1.58113883, -0.15075567, -0.60302269],
       [ 0.        ,  0.        ,  1.63090227,  0.15075567]])

Теперь можем применить полученный pipeline
Видим, что пропусков нет

In [ ]:
# res_num = pd.DataFrame(res_num, columns=pipe_num['scaler'].get_feature_names_out(np_Small_BAD_test_FD001[0,:]))
# res_num

res = pipe_num.transform(np_Small_BAD_test_FD001)

#### Сохраняем Pipline

In [18]:
savePath = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/final/Pipelinr.pickle'

with open(savePath, 'wb') as handle:
    save_pik_pipeline = pickle.dumps(pipe_num)

In [18]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'
save_dir = f"{inp_dir}/"
files = os.listdir(inp_dir)

files

['new_1.txt']

In [4]:
for file in files:
    new_name = f'{save_dir}new_{file}'
    os.rename(save_dir + file, new_name)

#### Сохраняем pickle

In [ ]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/tests'
out_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'

employ_Pipline(inp_dir,
               out_dir,
               pipe_num) 

### Преобразование data Frame

1. Добавляем столбец с индексами
2. Удаляем строку с названием столбцов
3. Удаляем строку, если в ней есть пропуск 

In [4]:
# 1 - Добавление столбца с индексами
def add_col_indexes(dataFrame: np.array) -> np.array:
    
    str_count, col_count = dataFrame.shape
    indexses = []
    indexses.append(0)

    print("str -->", str_count,
          "\ncol -->", col_count)
    
    for i in range(str_count-1):
        indexses.append(i)

    dataFrame = np.insert(dataFrame, 0, indexses, axis= 1)

    return dataFrame

In [5]:
# 2 - Удаление 0-й строки с названиями столбцов
def delete_col_names(dataFrame: np.array) -> np.array:
    dataFrame = dataFrame[1:, 0:]

    return dataFrame

In [41]:
def delete_nan_str(dataFrame: np.array) -> np.array:
    
    line, col  = dataFrame.shape
    res_dataFrame = np.empty((1, col))

    for line in dataFrame:

        line = np.array([line])
        print(line)

        if(check_nan_dataFrame(line) == False):
            res_dataFrame = np.vstack((res_dataFrame, line))
        else:
            continue

    res_dataFrame = delete_col_names(res_dataFrame)
    
    return res_dataFrame

delete_nan_str(np_Small_BAD_test_FD001)

[[nan nan nan nan]]
[[1.e+00 1.e+00    nan 3.e-04]]
[[ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]]
[[1.e+00 3.e+00 3.e-04    nan]]
[[1.        nan 0.0042 0.    ]]


array([[ 1.0e+00,  2.0e+00, -2.7e-03, -3.0e-04]])

In [8]:
def to_standardization_df(dataFrame: np.array) -> np.array:
    
    dataFrame = add_col_indexes(dataFrame)
    dataFrame = delete_col_names(dataFrame)
    dataFrame = delete_nan_str(dataFrame)

    return dataFrame

to_standardization_df(np_Small_BAD_test_FD001)

str --> 5 
col --> 4


array([[ 0.0e+00,  1.0e+00,  1.0e+00,  2.3e-03,  3.0e-04],
       [ 1.0e+00,  1.0e+00,  2.0e+00, -2.7e-03, -3.0e-04],
       [ 2.0e+00,  1.0e+00,  3.0e+00,  3.0e-04,  1.0e-04],
       [ 3.0e+00,  1.0e+00,  4.0e+00,  4.2e-03,  0.0e+00]])

### Из NumPy в Pandas

In [ ]:
values = np_Small_BAD_test_FD001[1:,1:].astype(float)
colums = np_Small_BAD_test_FD001[0, 1:]

pd_Small_BAD_test_FD001 = pd.DataFrame(values, columns = colums)
pd_Small_BAD_test_FD001

### Ф-ии проверки данных / наличия nan
Для указания строки используй --> df[str, :]

In [5]:
def out_info(status: bool, text: str):
    print(f"{text} result is {status}")

In [34]:
def is_nan_dataFrame_Line(dataset: np.array) -> bool: #

   #dataset_v = np.delete(dataset, (0), axis=0)
    status_log = ["Check Nan Line dataframe successfull", "Check Nan Line dataframe error"]
    res: bool = False

    for st in dataset:
        # print(f"{col} --> {type(col)}")
        if(np.isnan(st)):
            res = True
            out_info(res, status_log[0])
            return res
        
    out_info(res, status_log[1])
    return res

print(np_Small_BAD_test_FD001[2, :])
is_nan_dataFrame_Line(np_Small_BAD_test_FD001[2, :])

[ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]
Check Nan Line dataframe error result is False


False

In [7]:
def is_named_col(dataset: np.array) -> bool:

        status_log = ["Check first string dataframe successfull", "Check first string dataframe error"]
        res = False

        first_str = dataset[0, :]
        print(first_str)

        for val in first_str:
            if(np.isnan(val)):
                res = True
            else:
                res = False
                return res
        
        return res


is_named_col(np_Small_BAD_test_FD001)

[nan nan nan nan]


True

In [14]:
def delete_nan_str(dataFrame: np.array) -> np.array:
    
        status_log = ["Delete NULL lines in dataframe successfull", "Delete NULL lines in dataframe error"]
        status = False

        str, col  = dataFrame.shape
        print(f"str = {str}\ncol = {col}")
        res_dataFrame = np.zeros(col)

        for line in dataFrame:
            if(is_nan_dataFrame_Line(line) == False):
                res_dataFrame = np.vstack((res_dataFrame, line))
                print(f"res_dataFrame = {res_dataFrame}")
            else:
                continue

        res_dataFrame = res_dataFrame[1:, :]
        
        out_info(True, status_log[0])
        return res_dataFrame


res = delete_nan_str(np_Small_BAD_test_FD001)
print("res = ", res)

str = 5
col = 4
Check Nan Line dataframe successfull result is True
Check Nan Line dataframe successfull result is True
Check Nan Line dataframe error result is False
res_dataFrame = [[ 0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]]
Check Nan Line dataframe successfull result is True
Check Nan Line dataframe successfull result is True
Delete NULL lines in dataframe successfull result is True
res =  [[ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]]


In [13]:
def add_col_indexes(dataFrame: np.array) -> np.array:
        
        status_log = ["Add column with indexes in dataframe successfull", "Add column with indexes in dataframe error"]

        str_count, col_count = dataFrame.shape
        indexses = []
        coef_if_empty = 0

        print("str -->", str_count,
            "\ncol -->", col_count)

        if (is_nan_dataFrame_Line(dataFrame[0, :])):
            indexses.append(np.nan)
            coef_if_empty = 1
        
        for i in range(str_count - coef_if_empty):
            indexses.append(i)

        dataFrame = np.insert(dataFrame, 0, indexses, axis= 1)

        out_info(True, status_log[0])
        return dataFrame


print(GOOD_dataFrame)
print("res = \n", add_col_indexes(GOOD_dataFrame))

[[1 1 1]
 [1 1 1]
 [1 1 1]]
str --> 3 
col --> 3
Check Nan Line dataframe error result is False
Add column with indexes in dataframe successfull result is True
res = 
 [[0 1 1 1]
 [1 1 1 1]
 [2 1 1 1]]


In [35]:
def delete_names(dataFrame: np.array) -> np.array:
    
    status_log = ["Delete columns with names in dataframe successfull", "Delete columns with names in dataframe error"]

    if(is_named_col(dataFrame) == True):
        dataFrame = dataFrame[1:, 0:]

    out_info(True, status_log[0])
    return dataFrame

In [36]:
def to_standardization_df(dataFrame: np.array) -> np.array:

    status_log = ["Standartization dataframe successfull", "Standartization dataframe error"]

    dataFrame = add_col_indexes(dataFrame)
    dataFrame = delete_names(dataFrame)
    dataFrame = delete_nan_str(dataFrame)

    out_info(True, status_log[0])
    return dataFrame

### Функции для модуля

In [48]:
def employ_Pipline(inp_dir: str,       # B inpFilesList и outFilesList указывать полный путь
                    out_dir: str,
                    pipline: Pipeline = pipe_num) -> bool:
    
    status_log  =           ["Preprocess data finished successfull",        "Preprocess data finished error"]
    get_doc_log =           ["Getting a list of documents...",              "Documents have been received"]

    # На конце выходной строки дирректории должна стоять "/"
    if inp_dir[-1] != '/':
        inp_dir = f"{inp_dir}/"
    if out_dir[-1] != '/':
        out_dir = f"{out_dir}/"

    # Получаем все документы в папке
    print(get_doc_log[0])
    inpFilesList = os.listdir(inp_dir)
    outFilesList = inpFilesList
    
    # Вывод списка файлов в диррекории
    for fl in inpFilesList:
        print("   .../" + fl)
        
    print(get_doc_log[1])

    # Применение
    for file in inpFilesList:
        if file == ".gitkeep":
            continue
        
        print("  Processed --> ", file)

        # Загружаем i-тый фаил
        dataFrame = genfromtxt(inp_dir + file, delimiter = ',')
        
        # Проверяем датасет на пригодность (отсутствие пропусков)
        for line in dataFrame:
            if (is_nan_dataFrame_Line(line) == False):
                continue
            else:
                print("Dataset is BAD Starting standartization dataframe...")
                to_standardization_df(dataFrame)
                break
        
        print("Dataset is GOOD  Starting employ pipeline...")
        
        # newDataFrame = pd.DataFrame(dataFrame, columns = pipline['scaler'].get_feature_names_out(dataFrame.columns))
        newDataFrame = pipline.fit_transform(dataFrame)

        # Сохранение
        fileName = out_dir + "New_" + file + ".csv"
        np.savetxt(fileName, newDataFrame, delimiter = ",")
    
    out_info(True, status_log[0])
    return True




inp_dir = General_path + raw + tests
out_dir = General_path + processed

employ_Pipline(inp_dir,
                out_dir)

Getting a list of documents...
   .../Small_BAD_test_FD001.csv
   .../Small_GOOD_test_FD001.csv
Documents have been received
  Processed -->  Small_BAD_test_FD001.csv
Check Nan Line dataframe successfull result is True
Dataset is BAD Starting standartization dataframe...
str --> 5 
col --> 4
Check Nan Line dataframe successfull result is True
Add column with indexes in dataframe successfull result is True
[nan nan nan nan nan]
Delete columns with names in dataframe successfull result is True
str = 4
col = 5
Check Nan Line dataframe successfull result is True
Check Nan Line dataframe error result is False
res_dataFrame = [[ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]]
Check Nan Line dataframe successfull result is True
Check Nan Line dataframe successfull result is True
Delete NULL lines in dataframe successfull result is True
Standartization dataframe successfull result is True
Dataset is GOOD  Starting employ pipeline...
  Processed -->

True

### Разделение датасета на нормальные и аномальные значения

In [11]:
def differentiation_on_anomaly(dataFrame: np.array,
                               last_anomaly: int,
                               out_dir: str,
                               file_name: str):
    
    count_unit_number = dataFrame[1, 0]
    count_time_cycles = 1

    current_str_num = 0
    count = 0

    last_time_cycles = []

    str, col  = dataFrame.shape

    normal_df = np.zeros(col)
    anomalmal_df = np.zeros(col)

    unit_number = 0

    fileName_Normal_DF = out_dir + "New_NORMAL" + file_name + ".csv"
    fileName_Anomal_DF = out_dir + "New_ANOMAL" + file_name + ".csv"
    
    for str in dataFrame:
        unit_number = dataFrame[current_str_num, 0]
        #print(unit_number)

        if(count_unit_number != unit_number):
            last_time_cycles.append(count_time_cycles)
            count_unit_number = unit_number
        
        count_time_cycles += 1
        current_str_num += 1

    print(last_time_cycles)

    count_unit_number = dataFrame[1, 0]
    count_time_cycles = 1
    current_str_num = 0

    for str in dataFrame:

        unit_number = dataFrame[current_str_num, 0]

        if(count_unit_number != unit_number):
            count += 1
            count_unit_number = unit_number

        if(count_time_cycles < (last_time_cycles[count - 1] - last_anomaly)):
            print(count, last_time_cycles[count - 1])
            normal_df = np.vstack((normal_df, str))
        
        else: 
            anomalmal_df = np.vstack((anomalmal_df, str))

        current_str_num += 1


    # normal_df = normal_df[1:, :]
    # anomalmal_df = anomalmal_df[1:, :]


    # np.savetxt(fileName_Normal_DF, normal_df, delimiter = ",")
    # np.savetxt(fileName_Anomal_DF, anomalmal_df, delimiter = ",")




out_dir = General_path + processed
last_anomaly = 50
file_name = "New_train_FD001"

differentiation_on_anomaly(New_train_FD001,
                           last_anomaly,
                           out_dir,
                           file_name)

[193, 480, 659, 848, 1117, 1305, 1564, 1714, 1915, 2137, 2377, 2547, 2710, 2890, 3097, 3306, 3582, 3777, 3935, 4169, 4364, 4566, 4734, 4881, 5111, 5310, 5466, 5631, 5794, 5988, 6222, 6413, 6613, 6808, 6989, 7147, 7317, 7511, 7639, 7827, 8043, 8239, 8446, 8638, 8796, 9052, 9266, 9497, 9712, 9910, 10123, 10336, 10531, 10788, 10981, 11256, 11393, 11540, 11771, 11943, 12128, 12308, 12482, 12765, 12918, 13120, 13433, 13632, 13994, 14131, 14339, 14552, 14765, 14931, 15160, 15370, 15524, 15755, 15954, 16139, 16379, 16593, 16886, 17153, 17341, 17619, 17797, 18010, 18227, 18381, 18516, 18857, 19012, 19270, 19553, 19889, 20091, 20247, 20432]
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432
0 20432


17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
17 3582
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777
18 3777


KeyboardInterrupt: 

### Источники
1 - https://habr.com/ru/companies/yandex_praktikum/articles/756474/